## Instalar namesDataset

In [ ]:
!pip install names-dataset

## Preprocesado datos del INE


In [ ]:
import pandas as pd
from google.colab import files, drive
import io

excel_file_path = "nombres_mas_frecuentes.xlsx"
nombre_archivo = excel_file_path
sheet_name = 3

df_full = pd.read_excel(
    nombre_archivo,
    sheet_name=sheet_name,
    header=None
)

provinces_series_raw = df_full.iloc[2]
provinces_series_filled = provinces_series_raw.ffill()

df_data = pd.read_excel(
    nombre_archivo,
    sheet_name=sheet_name,
    header=3
)

dfs_list = []

for i in range(1, len(df_data.columns),3):
    temp_df = df_data.iloc[:, i:i+3].copy()

    if temp_df.shape[1] == 3:
        province = df_data.columns[i]
        temp_df.columns = ['Nombre', 'Frecuencia', 'Por 1.000']

        temp_df['Provincia'] = province

        dfs_list.append(temp_df)

nombres_hombres = pd.concat(dfs_list, ignore_index=True)
nombres_hombres.dropna(subset=['Nombre'], inplace=True)
nombres_hombres = nombres_hombres[nombres_hombres['Nombre'] != 'NOMBRE']
nombres_hombres.reset_index(drop=True, inplace=True)

nombres_hombres['Frecuencia'] = pd.to_numeric(nombres_hombres['Frecuencia'], errors='coerce')
nombres_hombres['Por 1.000'] = pd.to_numeric(nombres_hombres['Por 1.000'], errors='coerce')

nombres_hombres['Provincia'] = nombres_hombres['Provincia'].astype(str)

print("\nDataFrame final reestructurado (primeras filas):")
print(nombres_hombres.head())
print("\nNúmero de filas en el DataFrame final:", len(nombres_hombres))

In [ ]:
import pandas as pd
from google.colab import files, drive
import io

excel_file_path = "nombres_mas_frecuentes.xlsx"
nombre_archivo = excel_file_path
sheet_name = 4

df_full = pd.read_excel(
    nombre_archivo,
    sheet_name=sheet_name,
    header=None
)

provinces_series_raw = df_full.iloc[2]
provinces_series_filled = provinces_series_raw.ffill()

df_data = pd.read_excel(
    nombre_archivo,
    sheet_name=sheet_name,
    header=3
)

dfs_list = []

for i in range(1, len(df_data.columns),3):
    temp_df = df_data.iloc[:, i:i+3].copy()

    if temp_df.shape[1] == 3:
        province = df_data.columns[i]
        temp_df.columns = ['Nombre', 'Frecuencia', 'Por 1.000']

        temp_df['Provincia'] = province

        dfs_list.append(temp_df)

nombres_mujeres = pd.concat(dfs_list, ignore_index=True)
nombres_mujeres.dropna(subset=['Nombre'], inplace=True)
nombres_mujeres = nombres_mujeres[nombres_mujeres['Nombre'] != 'NOMBRE']
nombres_mujeres.reset_index(drop=True, inplace=True)

nombres_mujeres['Frecuencia'] = pd.to_numeric(nombres_mujeres['Frecuencia'], errors='coerce')
nombres_mujeres['Por 1.000'] = pd.to_numeric(nombres_mujeres['Por 1.000'], errors='coerce')

nombres_mujeres['Provincia'] = nombres_mujeres['Provincia'].astype(str)

print("\nDataFrame final reestructurado (primeras filas):")
print(nombres_mujeres.head())
print("\nNúmero de filas en el DataFrame final:", len(nombres_mujeres))

In [ ]:
nombres_comunidad = pd.concat([nombres_hombres, nombres_mujeres], ignore_index=True)
nombres_comunidad

In [ ]:
import pandas as pd
from google.colab import files, drive
import io

excel_file_path = "apellidos_mas_frecuentes.xls"
nombre_archivo = excel_file_path
sheet_name = 2

df_full = pd.read_excel(
    nombre_archivo,
    sheet_name=sheet_name,
    header=None
)

provinces_series_raw = df_full.iloc[2]
provinces_series_filled = provinces_series_raw.ffill()

df_data = pd.read_excel(
    nombre_archivo,
    sheet_name=sheet_name,
    header=3
)

dfs_list = []

for i in range(1, len(df_data.columns),3):
    temp_df = df_data.iloc[:, i:i+3].copy()

    if temp_df.shape[1] == 3:
        province = df_data.columns[i]
        temp_df.columns = ['Apellido', 'Frecuencia', 'Por 1.000']

        temp_df['Provincia'] = province

        dfs_list.append(temp_df)

apellidos = pd.concat(dfs_list, ignore_index=True)
apellidos.dropna(subset=['Apellido'], inplace=True)
apellidos = apellidos[apellidos['Apellido'] != 'PRIMER APELLIDO']
apellidos.reset_index(drop=True, inplace=True)

apellidos['Frecuencia'] = pd.to_numeric(apellidos['Frecuencia'], errors='coerce')
apellidos['Por 1.000'] = pd.to_numeric(apellidos['Por 1.000'], errors='coerce')

apellidos['Provincia'] = apellidos['Provincia'].astype(str)

print("\nDataFrame final reestructurado (primeras filas):")
print(apellidos.head())
print("\nNúmero de filas en el DataFrame final:", len(apellidos))

## Predictor

In [ ]:
import pandas as pd
from names_dataset import NameDataset
from collections import defaultdict
import numpy as np

def get_most_probable_country(scores_dict):
    """Dado un diccionario de {pais: puntuacion}, retorna el país con la puntuación más alta."""
    filtered_scores = {k: v for k, v in scores_dict.items() if v > 0}
    if not filtered_scores:
        return "Desconocido"
    try:
        most_probable = max(filtered_scores, key=filtered_scores.get)
        return most_probable
    except ValueError:
        return "Desconocido"

def predict_autonomous_community(first_name, df_comunidad):
    """
    Predice la Comunidad Autónoma probable basada en el nombre de pila y el género,
    usando los datos del INE.
    """

    nombre_filtered_df = df_comunidad[df_comunidad['Nombre'].str.upper() == first_name.upper()]

    if nombre_filtered_df.empty:
        return "Desconocido (Nombre no encontrado en INE)"

    community_scores = defaultdict(float)
    for index, row in nombre_filtered_df.iterrows():
        provincia_str = row['Provincia']
        score_por_mil = row['Por 1.000']

        if not pd.isna(score_por_mil):
             community_scores[provincia_str] += score_por_mil
        else:
             print(f"Advertencia: Score NaN para '{first_name}' en '{provincia_str}'. Saltando.")


    return get_most_probable_country(community_scores)

def predict_autonomous_community_surname(surname, df_comunidad):
    """
    Predice la Comunidad Autónoma probable basada en el nombre de pila y el género,
    usando los datos del INE.
    """

    nombre_filtered_df = df_comunidad[df_comunidad['Apellido'].str.upper() == surname.upper()]

    if nombre_filtered_df.empty:
        return "Desconocido (Apellido no encontrado en INE)"

    community_scores = defaultdict(float)
    for index, row in nombre_filtered_df.iterrows():
        provincia_str = row['Provincia']
        score_por_mil = row['Por 1.000']

        if not pd.isna(score_por_mil):
             community_scores[provincia_str] += score_por_mil
        else:
             print(f"Advertencia: Score NaN para '{surname}' en '{provincia_str}'. Saltando.")


    return get_most_probable_country(community_scores)


def parse_name_parts(full_name):
    """
    Intenta dividir un nombre completo en nombre(s) de pila y apellidos(s)
    basándose en una heurística simple.
    Puede necesitar ajustes para diferentes convenciones.
    """
    parts = full_name.strip().split()
    num_parts = len(parts)

    if num_parts == 0:
        return None, None, None

    nombre = parts[0]
    apellido1 = None
    apellido2 = None

    if num_parts == 1:
        pass
    elif num_parts == 2:
        apellido1 = parts[1]
    elif num_parts == 3:
        nombre = parts[0]
        apellido1 = parts[1]
        apellido2 = parts[2]
    else:
        pos = 0
        nombre_completo = ["", "", ""]
        parts.reverse()
        for part in range(0, num_parts):
            if len(parts[part]) > 3:
              if nombre_completo[pos] == "":
                nombre_completo[pos] = parts[part]
                pos += 1
              else:
                nombre_completo[pos] = nombre_completo[pos] + " " + parts[part]
            elif len(parts[part]) <= 3:
                nombre_completo[pos - 1] = parts[part] + " " + nombre_completo[pos - 1]
            if pos >= 2:
              pos = 2
        apellido2 = nombre_completo[0]
        apellido1 = nombre_completo[1]
        nombre = nombre_completo[2]
        print(f"Nombre: {nombre}; Primer apellido: {apellido1}; Segundo apellido: {apellido2}")


    print(f"Nombre: {nombre}; Primer apellido: {apellido1}; Segundo apellido: {apellido2}")
    return nombre, apellido1, apellido2


def process_customer_name_improved(nd_instance, full_name):
    """
    Procesa una cadena de nombre completo usando parsing mejorado
    y combinación de puntuaciones ponderada.
    """
    if not isinstance(full_name, str) or not full_name.strip():
        return "Desconocido", "Desconocido", "Desconocido", "Desconocido"

    cleaned_name = full_name.strip()

    nombre, apellido1, apellido2 = parse_name_parts(cleaned_name)

    weight_nombre = 0.3
    weight_apellido1 = 0.4
    weight_apellido2 = 0.3

    scores_nombre = {}
    scores_s1 = {}
    scores_s2 = {}

    pais_nombre_probable = "Desconocido"
    if nombre:
        try:
            search_result = nd_instance.search(nombre)
            if 'first_name' in search_result and 'country' in search_result['first_name']:
                scores_nombre = search_result['first_name']['country']

            pais_nombre_probable = get_most_probable_country(scores_nombre)
        except Exception as e:
            pass


    pais_s1_probable = "Desconocido"
    if apellido1:
        try:
            search_result = nd_instance.search(apellido1)
            if 'last_name' in search_result and 'country' in search_result['last_name']:
                 scores_s1 = search_result['last_name']['country']
            pais_s1_probable = get_most_probable_country(scores_s1)
        except Exception as e:
            pass

    pais_s2_probable = "Desconocido"
    if apellido2:
          try:
            search_result = nd_instance.search(apellido2)
            if 'last_name' in search_result and 'country' in search_result['last_name']:
                 scores_s2 = search_result['last_name']['country']
            pais_s2_probable = get_most_probable_country(scores_s2)
          except Exception as e:
            pass

    combined_scores = defaultdict(float)

    if scores_nombre:
        for country, score in scores_nombre.items():
            combined_scores[country] += score * weight_nombre

    if scores_s1:
        for country, score in scores_s1.items():
            combined_scores[country] += score * weight_apellido1

    if scores_s2:
        for country, score in scores_s2.items():
            combined_scores[country] += score * weight_apellido2

    pais_final_probable = get_most_probable_country(combined_scores)

    return pais_nombre_probable, pais_s1_probable, pais_s2_probable, pais_final_probable


print("Inicializando NamesDataset. Esto puede tardar un poco la primera vez...")
try:
    from names_dataset import NameDataset
    nd = NameDataset()
    print("NamesDataset inicializado correctamente.")
except ImportError:
    try:
        from names_dataset import NamesDataset
        nd = NamesDataset()
        print("NamesDataset inicializado correctamente (importación alternativa).")
    except Exception as e:
        print(f"Error fatal al inicializar NamesDataset: {e}")
        print("Por favor, asegúrate de que la librería 'names-dataset' está instalada correctamente.")
        exit()


archivo_clientes = 'Clientes_Tienda.xlsx'
columna_razon_social = 'Razón social'

try:
    df = pd.read_excel(archivo_clientes)

    if columna_razon_social not in df.columns:
        raise ValueError(f"La columna '{columna_razon_social}' no se encontró en el archivo.")

    print(f"Dataset '{archivo_clientes}' cargado correctamente. Filas encontradas: {len(df)}")
    print(f"Utilizando la columna '{columna_razon_social}' para extraer nombres.")

except FileNotFoundError:
    print(f"Error: Archivo '{archivo_clientes}' no encontrado. Por favor, verifica la ruta.")
    exit()
except ValueError as ve:
     print(f"Error en el dataset: {ve}")
     exit()
except Exception as e:
    print(f"Error al cargar el dataset: {e}")
    exit()


paises_nombre_list = []
paises_s1_list = []
paises_s2_list = []
paises_final_list = []

print("Procesando clientes para determinar países probables...")

for index, row in df.iterrows():
    full_name = row[columna_razon_social]

    pais_nombre, pais_s1, pais_s2, pais_final = process_customer_name_improved(nd, full_name)
    name_parts = full_name.strip().split()
    spain = 0

    comunidad_probable = predict_autonomous_community(
        full_name.strip().split()[0],
        nombres_comunidad
    )
    if comunidad_probable == 'Desconocido (Nombre no encontrado en INE)':
          comunidad_probable = pais_nombre
    else:
      spain += 1

    comunidad_probable2 = predict_autonomous_community_surname(full_name.strip().split()[1], apellidos)
    if comunidad_probable2 == 'Desconocido (Apellido no encontrado en INE)':
      comunidad_probable2 = pais_s1
    else:
      spain += 1

    comunidad_probable3 = pais_s2
    if len(name_parts) > 2:
        comunidad_probable3 = predict_autonomous_community_surname(name_parts[2], apellidos)
        if comunidad_probable3 == 'Desconocido (Apellido no encontrado en INE)':
            comunidad_probable3 = pais_s2
        else:
          spain += 1


    paises_nombre_list.append(comunidad_probable)
    paises_s1_list.append(comunidad_probable2)
    paises_s2_list.append(comunidad_probable3)
    if spain >= 2:
      paises_final_list.append('Spain')
    else:
      paises_final_list.append(pais_final)

    if (index + 1) % 100 == 0:
        print(f"--- {index + 1} clientes procesados ---")

print("Procesamiento de clientes completado.")

df['País Nombre Probable'] = paises_nombre_list
df['País 1er Apellido Probable'] = paises_s1_list
df['País 2do Apellido Probable'] = paises_s2_list
df['País Final Probable'] = paises_final_list

print("\n--- DataFrame con países probables (primeras filas) ---")
columnas_a_mostrar = [columna_razon_social, 'País Nombre Probable', 'País 1er Apellido Probable', 'País 2do Apellido Probable', 'País Final Probable']
print(df[columnas_a_mostrar].head())

ruta_guardar = 'clientes_con_pais_probable.csv'
df.to_csv(ruta_guardar, index=False)
print(f"\nDataFrame actualizado guardado en '{ruta_guardar}'")